In [1]:
import zarr
import numpy as np
#Loading Required Libraries 
import anndata
import scipy as sp
import geopandas
#import squidpy as sq
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as dd
import dask.array as da
import dask.bag as db
import pandas as pd
from libpysal.weights import Queen, Rook, KNN
from esda.moran import Moran
from splot.esda import plot_moran
import pyseq
from pyseq import image_analysis as ia
import dask
#dask.config.set({'distributed.worker.multiprocessing-method': 'spawn'})
import warnings
warnings.filterwarnings('ignore')
from dask.distributed import Client
import xarray as xr
import dask
%run /gpfs/commons/home/jsingh/util_pyseq.ipynb

/gpfs/commons/home/jsingh/.conda/envs/spatial/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1dev-CAPI-1.14.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
#dask.config.set({'distributed.worker.multiprocessing-method': 'forkserver'})
dask.config.config

{'temporary-directory': None,
 'visualization': {'engine': None},
 'tokenize': {'ensure-deterministic': False},
 'dataframe': {'shuffle-compression': None,
  'parquet': {'metadata-task-size-local': 512,
   'metadata-task-size-remote': 16}},
 'array': {'svg': {'size': 120},
  'slicing': {'split-large-chunks': None},
  'chunk-size': '128MiB',
  'rechunk-threshold': 4},
 'optimization': {'fuse': {'active': None,
   'ave-width': 1,
   'max-width': None,
   'max-height': inf,
   'max-depth-new-edges': None,
   'subgraphs': None,
   'rename-keys': True}},
 'distributed': {'version': 2,
  'scheduler': {'allowed-failures': 3,
   'bandwidth': 100000000,
   'blocked-handlers': [],
   'contact-address': None,
   'default-data-size': '1kiB',
   'events-cleanup-delay': '1h',
   'idle-timeout': None,
   'transition-log-length': 100000,
   'events-log-length': 100000,
   'work-stealing': True,
   'work-stealing-interval': '100ms',
   'worker-ttl': '5 minutes',
   'pickle': True,
   'preload': [],
   

In [3]:
im = ia.get_HiSeqImages(image_path = 'zarrs/m387ntga2.zarr')

ImageAnalysis::Opened m387ntga2 


In [4]:
im.im

<xarray.DataArray 'm387ntga2' (obj_step: 10, channel: 4, cycle: 4, row: 10467,
                               col: 10752)>
dask.array<getitem, shape=(10, 4, 4, 10467, 10752), dtype=int16, chunksize=(1, 1, 1, 10467, 1792), chunktype=numpy.ndarray>
Coordinates:
  * channel   (channel) int64 558 610 687 740
  * cycle     (cycle) int64 1 2 3 4
  * obj_step  (obj_step) int64 8028 8263 8498 8733 8968 ... 9438 9673 9908 10143
Dimensions without coordinates: row, col
Attributes:
    first_group:  0
    machine:      None
    scale:        1
    overlap:      256
    fixed_bg:     1

In [5]:
#format = one_z_plane_obj_step_channel_cycle
array_object_list = []
for i in im.im['channel'].values:
    for j in im.im['cycle'].values:
        #for k #in im.im['obj_step'].values[1]:
            k = 8028
            nme = "one_x_plane_"+str(i)+"_"+str(j)+"_"+str(k)
            nme = im.im.sel(obj_step = k, cycle=j, channel = i)
            array_object_list.append(nme)
        


In [1]:
array_object_list[0]

NameError: name 'array_object_list' is not defined

In [6]:
import skimage

one_z_plane = im.im.sel(obj_step = 8263, channel = 610, cycle=1)
#gdf = geopandas.read_file('~/voronoi_regions/voronoi_regions.shp')
labels = skimage.io.imread('/gpfs/commons/groups/nygcfaculty/PySeq/20210428_mouse_genotype_2/segmented_sections/m387ntga2_labels.tiff')

In [ ]:
#!dask-scheduler --port 8000
#!dask-worker --dashboard-address 8000 --nanny-port 8001

In [9]:
def get_cluster(queue_name = 'pe2', log_dir=None):
    """ Make dask cluster w/ workers = 2 cores, 32 G mem, and 1 hr wall time.

        return cluster, client
    """

    if log_dir is None:
        log_dir = join(getcwd(),'dask_logs')
        makedirs(log_dir, exist_ok=True)

    cluster = SLURMCluster(
                queue = queue_name,
                cores = 2,
                memory = '32G',
                walltime='1:00:00',
                log_directory=log_dir,
                extra=["--lifetime", "55m", "--lifetime-stagger", "4m"])
    client = Client(cluster)

    return cluster, client

cluster, client = get_cluster()
print(cluster.dashboard_link)

http://10.4.202.122:8787/status


In [ ]:
#cluster.close()

In [10]:
cluster.scale(cores = 10)

In [ ]:
one_z_plane

In [7]:
%%time
n_labels = labels.max()
mean_intensity = []
for l in range(10):
    mean_intensity.append(one_z_plane.data[labels == l+1].mean())

CPU times: user 1.97 s, sys: 235 ms, total: 2.21 s
Wall time: 2.2 s


In [8]:
n_labels

11232

In [ ]:
pixel_array = []
#pixel_array = dask.array.from_array(pixel_array)
pixel_array = np.asarray(pixel_array)
def get_selected(lab):
    ar  = np.append(pixel_array,[one_z_plane.data[labels == lab+1]])
    return ar

In [ ]:
get_selected = delayed(get_selected)

In [ ]:
#%%time
#s = get_selected(2)
#s.compute()

In [ ]:
def gget_selected(lab):
    ar  = pixel_array,[one_z_plane.values[labels == lab+1]]
    return ar


In [ ]:
%%time
gget_selected(2)

In [ ]:
gget_selected(1).compute()

In [11]:
persisted_plane = array_object_list[0].persist()

In [12]:
persisted_plane

<xarray.DataArray 'm387ntga2' (row: 10467, col: 10752)>
dask.array<getitem, shape=(10467, 10752), dtype=int16, chunksize=(10467, 1792), chunktype=numpy.ndarray>
Coordinates:
    channel   int64 558
    cycle     int64 1
    obj_step  int64 8028
Dimensions without coordinates: row, col
Attributes:
    first_group:  0
    machine:      None
    scale:        1
    overlap:      256
    fixed_bg:     1

In [ ]:
## VIA VALUES 
def get_pixels(lab):
    m = one_z_plane.values[labels == lab+1].mean()
    return m

In [13]:
def get_pixels_persisted(lab,plane):
    m = plane.values[labels == lab+1].mean()
    return m

In [15]:
def get_pixels_persisted(lab):
    m = persisted_plane.values[labels == lab+1].mean()
    return m


In [ ]:
%%time
s = get_pixels_persisted(lab = 23,plane = persisted_plane )
s

In [ ]:
for lab,plane in zip(range(10),persisted_plane):
    print(lab,plane.data)

In [ ]:
from joblib import Parallel, delayed
    
results = Parallel(n_jobs=2)(delayed(get_pixels_persisted)(lab) for lab in range(2))
print(results) 

In [16]:
cluster.scheduler._address

'tcp://10.4.202.122:35374'

In [17]:
%%time
from joblib import Parallel, delayed
from joblib import parallel_backend
with parallel_backend('dask',scheduler_host='tcp://10.4.202.122:35374'):
    results = Parallel(n_jobs=-2)(delayed(get_pixels_persisted)(lab) for lab in range(11232))
    

KeyboardInterrupt: 

In [ ]:
%%time
from joblib import Parallel, delayed
from joblib import parallel_backend
with parallel_backend('dask',scheduler_host='tcp://10.4.202.105:41034'):
    results = Parallel(n_jobs=-1)(delayed(get_pixels)(lab) for lab in range(400))
    

In [18]:
i = 5

In [ ]:
results

In [19]:
len(results)

NameError: name 'results' is not defined

In [ ]:
pixel_array = []
#pixel_array = np.asarray(pixel_array)
@delayed
def gget_selected(lab):
    ar  = one_z_plane.values[labels == lab+1].mean()
    return ar

for l in range(10000):
    #ar2 = gget_selected(l)
    #mean_intensity = ar2.mean()
    pixel_array.append(gget_selected(l))
    
total = delayed(pixel_array)
#total = delayed(mean)
#dask.compute(pixel_array)
total.compute()

In [ ]:
get_pixels_persisted(lab = 1, plane = persisted_plane)

In [ ]:
pixel_array = []
#pixel_array = np.asarray(pixel_array)
def get_pixels_persisted(lab,plane):
    m = plane.values[labels == lab+1].mean()
    return m

for l in range(10000):
    print(l)
    pixel_array.append(get_pixels_persisted(lab = l, plane = persisted_plane))
    
#total = delayed(pixel_array)
#total = delayed(mean)
#dask.compute(pixel_array)
#total.compute()

In [ ]:
dask.compute(pixel_array[1])

In [ ]:
total.compute()

In [ ]:
total

In [ ]:
get_pixels = delayed(get_pixels)

In [ ]:
ss = get_pixels(3)

In [ ]:
ss.compute()

In [ ]:
mean_intensity = []
for l in range(10):
    m = get_pixels(l)
    mean_intensity.append(m)

In [ ]:
m

In [ ]:
m = one_z_plane.values[labels == 2].mean()

In [ ]:
[labels == 3]

In [ ]:
da.from_delayed(mean_intensity, shape=(10,1))

In [ ]:
type(one_z_plane)

In [ ]:
one_z_plane =  one_z_plane.chunk(chunks = (6,6))

In [ ]:
%prun dask.compute(mean_intensity)

In [ ]:
%prun one_z_plane.data[labels == l+1]

In [ ]:
%%time
one_z_plane.data[labels == l+1].mean()

In [ ]:
%%time
one_z_plane.values[labels == l+1].mean()

In [ ]:
from dask import delayed

In [ ]:
mean_intensity = []
def get_pixels(lab):
    m = one_z_plane.values[labels == lab+1].mean()
    mean_intensity.append(m)

In [ ]:
get_pixels = delayed(get_pixels)

In [ ]:
s = get_pixels(9)

In [ ]:
s.compute()

In [ ]:
from math import sqrt
from joblib import Parallel, delayed

In [ ]:
Parallel(n_jobs=2)(delayed(get_pixels)(l) for l in range(2))

In [ ]:
m = one_z_plane.values[labels == lab+1].mean()

In [ ]:
import dask
from dask import delayed

In [ ]:
%%time

list_names=['a','b','c','d']
keep_return=[]

@delayed
def loop_dummy(target):
    for i in range (1000000000):
        pass
    print('passed value is:'+target)
    return(1)


for i in list_names:
    c=loop_dummy(i)
    keep_return.append(c)


total = delayed(keep_return)
total.compute()

In [ ]:
keep_return[0].compute()